## 패키지

In [38]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from google.colab import drive

In [39]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/03_공모전/02_DACON/02_임대주택 단지 내 적정 주차수요 예측 AI 경진대회/')

## 데이터

In [41]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [42]:
print(train.shape)
train.head()

(2952, 15)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [43]:
print(test.shape)
test.head()

(1022, 14)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [48]:
train = train.replace('-', np.NaN)
test = test.replace('-', np.NaN)

In [49]:
train = train.astype({'임대보증금':'float','임대료':'float'})
test = test.astype({'임대보증금':'float','임대료':'float'})

In [50]:
train2=train.corr(method='pearson')
train2

,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
총세대수,1.000000,0.042271,0.098205,0.068192,0.075493,0.099606,0.213843,-0.001446,0.501488,0.316703
전용면적,0.042271,1.000000,-0.035479,-0.089201,0.490950,0.740591,-0.013111,-0.007866,0.071260,0.115453
전용면적별세대수,0.098205,-0.035479,1.000000,0.166472,-0.099252,-0.057759,-0.056680,0.047112,0.281989,0.247085
공가수,0.068192,-0.089201,0.166472,1.000000,0.030728,-0.194322,-0.093334,0.043285,0.289819,0.118121
임대보증금,0.075493,0.490950,-0.099252,0.030728,1.000000,0.557144,0.027113,0.110757,0.270697,0.311469
임대료,0.099606,0.740591,-0.057759,-0.194322,0.557144,1.000000,0.071571,0.044654,0.279785,0.416536
도보 10분거리 내 지하철역 수(환승노선 수 반영),0.213843,-0.013111,-0.056680,-0.093334,0.027113,0.071571,1.000000,0.098380,-0.055544,-0.108592
도보 10분거리 내 버스정류장 수,-0.001446,-0.007866,0.047112,0.043285,0.110757,0.044654,0.098380,1.000000,0.070696,0.103752
단지내주차면수,0.501488,0.071260,0.281989,0.289819,0.270697,0.279785,-0.055544,0.070696,1.000000,0.859216
등록차량수,0.316703,0.115453,0.247085,0.118121,0.311469,0.416536,-0.108592,0.103752,0.859216,1.000000


## 결측치 확인

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [ ]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [ ]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

## 컬럼명 변경

지하철역 수와 버스 정류장 수의 컬럼명을 지하철, 버스로 변경하였습니다.

In [ ]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [ ]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

## 지역명 숫자로 매핑

In [ ]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i

In [ ]:
train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

## 전용면적을 5의 배수로 변경

In [ ]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

## 전용면적 상/하한 적용

상한100, 하한 15

In [ ]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [ ]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

## 단지별 데이터 1차원으로 취합

In [ ]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [ ]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:08, 48.90it/s]
150it [00:02, 51.81it/s]


In [ ]:
new_train

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,면적_30.0,면적_45.0,면적_40.0,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0,등록차량수
0,C2483,900.0,38.0,0.0,1425.0,0.0,3.0,149.0,665.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,C2515,545.0,17.0,1.0,624.0,0.0,3.0,80.0,132.0,0.0,276.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
2,C1407,1216.0,13.0,2.0,1285.0,1.0,1.0,0.0,124.0,0.0,390.0,340.0,362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0
3,C1945,755.0,6.0,3.0,734.0,1.0,3.0,240.0,303.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0
4,C1470,696.0,14.0,4.0,645.0,0.0,2.0,254.0,246.0,0.0,0.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,90.0,7.0,9.0,66.0,0.0,3.0,36.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0
419,C2035,492.0,24.0,5.0,521.0,0.0,1.0,156.0,0.0,0.0,0.0,180.0,0.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,246.0
420,C2020,40.0,7.0,8.0,25.0,1.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
421,C2437,90.0,12.0,11.0,30.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,16.0


## 결측치 처리

In [ ]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

## 학습

In [ ]:
new_train2=new_train.corr(method='pearson')
new_train2

,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,면적_30.0,면적_45.0,면적_40.0,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0,등록차량수
총세대수,1.000000,0.297797,-0.011684,0.741369,0.048283,0.103652,0.372495,0.135943,0.150685,0.387566,0.334011,0.182870,0.577765,0.082732,0.048724,0.252373,0.321400,0.048973,0.013127,0.018831,0.038394,0.579290
공가수,0.297797,1.000000,0.048573,0.380140,-0.040675,0.060399,0.257508,0.055074,-0.052022,-0.043239,0.220998,0.047219,0.033044,-0.037829,0.305379,0.236569,-0.133505,-0.051383,0.005814,-0.048802,-0.021771,0.167420
지역,-0.011684,0.048573,1.000000,0.013781,0.054326,-0.112137,0.022708,-0.149306,0.062050,-0.079082,-0.025392,-0.040796,-0.006220,0.133345,-0.024963,0.037655,-0.032434,-0.006843,-0.006095,0.123633,0.041230,0.032305
단지내주차면수,0.741369,0.380140,0.013781,1.000000,0.031472,0.090308,0.373729,0.239606,0.277357,0.067304,0.436240,0.078850,0.032462,0.233037,-0.000202,0.188467,-0.200433,-0.100895,0.024315,0.176841,0.093049,0.848199
지하철,0.048283,-0.040675,0.054326,0.031472,1.000000,0.082137,-0.065216,0.034859,0.006953,0.085086,0.078191,-0.035936,0.057158,-0.047424,-0.005988,-0.019504,-0.000967,0.010088,-0.011292,-0.070375,0.029600,0.028233
버스,0.103652,0.060399,-0.112137,0.090308,0.082137,1.000000,0.071064,0.132222,0.018396,-0.016149,0.042039,-0.030433,0.006033,-0.035173,-0.012269,0.009024,-0.000777,0.005815,0.061894,0.012937,-0.028740,0.107831
면적_35.0,0.372495,0.257508,0.022708,0.373729,-0.065216,0.071064,1.000000,-0.175160,-0.290713,-0.143161,0.435452,-0.039490,-0.019589,-0.190221,0.088954,0.134351,-0.155967,-0.065671,-0.065653,-0.209106,-0.062001,0.190600
면적_50.0,0.135943,0.055074,-0.149306,0.239606,0.034859,0.132222,-0.175160,1.000000,0.323093,-0.106166,-0.043923,-0.070336,-0.223528,-0.092284,-0.118820,-0.128850,-0.140959,-0.053901,-0.043380,-0.120376,-0.041007,0.267040
면적_55.0,0.150685,-0.052022,0.062050,0.277357,0.006953,0.018396,-0.290713,0.323093,1.000000,-0.137154,-0.184317,-0.071888,-0.156871,0.101273,-0.076927,-0.076716,-0.094978,-0.035671,0.008127,0.040604,-0.023353,0.387093
면적_30.0,0.387566,-0.043239,-0.079082,0.067304,0.085086,-0.016149,-0.143161,-0.106166,-0.137154,1.000000,-0.100429,0.167423,0.507436,-0.070384,-0.060302,0.009740,0.546403,0.181256,-0.015096,-0.077725,-0.023072,-0.097873


In [ ]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [ ]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

## 추론 및 제출

In [ ]:
pred = model.predict(x_test)

In [ ]:
submission['num'] = pred

In [ ]:
submission.to_csv('baseline.csv', index=False)